In [1]:
import os
import sys
path = !echo ${VIRTUAL_ENV}
path = os.path.join(path[0], '..')
sys.path.append(path)
from helper.utils import load_from_pickle, load_from_shelve
import numpy as np
import pandas as pd
import collections
# from plotly import tools
# import plotly as py
# import plotly.graph_objs as go
#import impywidgets as widget
# py.offline.init_notebook_mode(connected=True)

/home/jackalhan/Development/github/more_meaningful_representations/venv/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [57]:
base_data_path = "/home/jackalhan/Development/github/more_meaningful_representations/squad/train/improvement/model_performances/subset_5000"

In [61]:
debug_files = {}
for i, structure in enumerate(os.walk(base_data_path)):
    root, dirs, files = structure
    for file in files:
        if file.endswith(".pkl"):            
            debug_files[i] = os.path.join(root, file)
["{} --> {}".format(key, value) for key, value in debug_files.items()]

['1 --> /home/jackalhan/Development/github/more_meaningful_representations/squad/train/improvement/model_performances/subset_5000/model_2_layers_2_epoch_1_mar_1.0_sf_0.05_0.05_wd_0.001_0.001_lr_0.001_dim_8192_8192_keep_1_1_seed_66_66_loss_v3/debug_dict.pkl',
 '2 --> /home/jackalhan/Development/github/more_meaningful_representations/squad/train/improvement/model_performances/subset_5000/model_2_layers_2_epoch_1_mar_1.0_sf_0.05_0.05_wd_0.001_0.001_lr_0.001_dim_1024_1024_keep_1_1_seed_66_66_loss_v3/debug_dict.pkl',
 '3 --> /home/jackalhan/Development/github/more_meaningful_representations/squad/train/improvement/model_performances/subset_5000/model_2_layers_2_epoch_1_mar_1.0_sf_1_1_wd_0.001_0.001_lr_0.001_dim_7000_7000_keep_1_1_seed_66_66_loss_v3/debug_dict.pkl',
 '6 --> /home/jackalhan/Development/github/more_meaningful_representations/squad/train/improvement/model_performances/subset_5000/model_2_layers_2_epoch_1_mar_1.0_sf_1_1_wd_0.001_0.001_lr_0.001_dim_2048_4096_keep_1_1_seed_66_66_l

In [62]:
model_indx =2
my_dict = load_from_pickle(os.path.join(base_data_path, 
                                        debug_files[model_indx]))
num_of_epochs = my_dict['epochs']

In [63]:
eval_metrics = ['recall', 'precision', 'map', 'dcg', 'ndcg', 'arp', 'mrp']
top_k = ['eval_metric_name', '1', '2', '3', '5', '10', '20', '50', 'all']

#extract feature names and k into the hiarchical format
eval_metrics = {}
for key, value in my_dict['1'].items():
    _ = key.split('_')
    fn_name = _[0]
    k = _[1]
    if fn_name not in eval_metrics:
        eval_metrics[fn_name]={}
    if k == 'all':
        eval_metrics[fn_name][1000] = value
    else:
        eval_metrics[fn_name][int(k)] = value

# now place them to the dataframes
dataset = []
for fn_name, ks in eval_metrics.items():
    ks_ = collections.OrderedDict(sorted(ks.items()))
    row = []
    for k in top_k:
        if k == 'eval_metric_name':
            row.append(fn_name)
            continue
        if k == 'all':
            _k = 1000
        else:
            _k = int(k)
        if _k in ks_:
            row.append(ks[int(_k)])
        else:
            row.append('-')
    dataset.append(row)
    #print(fn_name)
df_dataset = pd.DataFrame(data=dataset, columns=top_k)
df_dataset

,eval_metric_name,1,2,3,5,10,20,50,all
0,recall,0.151376,0.219528,0.27654,0.344037,0.432503,0.539974,0.706422,-
1,precision,0.151376,0.109764,0.09218,0.0688073,0.0432503,0.0269987,0.0141284,-
2,mrp,-,-,-,-,-,-,-,0.247589
3,map,0.151376,0.185452,0.204456,0.219954,0.231751,0.239195,0.244596,-
4,dcg,0.151376,0.194375,0.222881,0.250744,0.279343,0.306497,0.339703,-
5,ndcg,0.151376,0.194375,0.222881,0.250744,0.279343,0.306497,0.339703,-
6,arp,-,-,-,-,-,-,-,42.6972
